### Config

In [24]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Asset

In [ ]:
a = Asset(coins.ETH, coins.BTC)
print(a.to_dict())
a = Asset.from_symbol('ETH/BTC')
print(a.to_dict())
a = Asset.from_symbol('ETHBTC')
print(a.to_dict())

### Balance

In [ ]:
b = Balance()
print("Currencies",  b.currencies)
print("Entries", b)
b.add_currency(coins.ETH)
b.update(coins.ETH, delta_free=1.0, delta_used=0.0)
print(b.get(coins.ETH))
b.get(coins.ETH)[BalanceType.FREE]
Balance.from_dict(b.to_dict())

### GDAX Client

In [25]:
from punisher.clients.gdax import gdax

In [22]:
async def main():
    trader = gdax.trader.Trader(product_id='BTC-USD')
    res = await asyncio.gather(
        trader.get_products(),
        trader.get_product_ticker(),
        trader.get_time(),
    )
    print(res)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

[[{'id': 'BCH-BTC', 'base_currency': 'BCH', 'quote_currency': 'BTC', 'base_min_size': Decimal('0.01'), 'base_max_size': Decimal('200'), 'quote_increment': Decimal('0.00001'), 'display_name': 'BCH/BTC', 'status': 'online', 'margin_enabled': False, 'status_message': None, 'min_market_funds': '0.001', 'max_market_funds': '30', 'post_only': False, 'limit_only': False, 'cancel_only': False}, {'id': 'BCH-USD', 'base_currency': 'BCH', 'quote_currency': 'USD', 'base_min_size': Decimal('0.01'), 'base_max_size': Decimal('350'), 'quote_increment': Decimal('0.01'), 'display_name': 'BCH/USD', 'status': 'online', 'margin_enabled': False, 'status_message': None, 'min_market_funds': '10', 'max_market_funds': '1000000', 'post_only': False, 'limit_only': False, 'cancel_only': False}, {'id': 'BTC-EUR', 'base_currency': 'BTC', 'quote_currency': 'EUR', 'base_min_size': Decimal('0.001'), 'base_max_size': Decimal('50'), 'quote_increment': Decimal('0.01'), 'display_name': 'BTC/EUR', 'status': 'online', 'margi

In [23]:
async def main():
    trader = gdax.trader.Trader(
        product_id='BTC-USD',
        api_key=cfg.GDAX_API_KEY,
        api_secret=cfg.GDAX_API_SECRET_KEY,
        passphrase=cfg.GDAX_PASSPHRASE)
    res = await trader.get_account()
    print(res)

    #res = await trader.buy(type='limit', size='0.01', price='2500.12')
    #print(res)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

[{'id': '842116a5-473a-46d8-b6b1-c1de6cddc815', 'currency': 'USD', 'balance': '4686.8754221916500000', 'available': '4686.87542219165', 'hold': '0.0000000000000000', 'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'}, {'id': 'ab22380c-b2df-4ff0-8902-f74b9336687c', 'currency': 'LTC', 'balance': '25.0000000000000000', 'available': '25', 'hold': '0.0000000000000000', 'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'}, {'id': '1a758fd9-b6fa-47ae-a004-be8e9602ca50', 'currency': 'BTC', 'balance': '1.6123827900000000', 'available': '1.61238279', 'hold': '0.0000000000000000', 'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'}, {'id': '2d41385a-80e4-476d-8ea8-93c3c0833a09', 'currency': 'ETH', 'balance': '0.0000000000000000', 'available': '0', 'hold': '0.0000000000000000', 'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'}, {'id': '708ce841-b717-46a8-9ef6-07ed4968da93', 'currency': 'BCH', 'balance': '0.0000000000000000', 'available': '0', 'hold': '0.0000000000000000', 'profile_id': '

In [ ]:
async def run_orderbook():
    async with gdax.orderbook.OrderBook(['ETH-USD', 'BTC-USD']) as orderbook:
        while True:
            message = await orderbook.handle_message()
            if message is None:
                continue
            print('ETH-USD ask: %s bid: %s' %
                  (orderbook.get_ask('ETH-USD'),
                   orderbook.get_bid('ETH-USD')))
            print('BTC-USD ask: %s bid: %s' %
                  (orderbook.get_ask('BTC-USD'),
                   orderbook.get_bid('BTC-USD')))

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(run_orderbook())

### Trade Feed

In [13]:
## Binance ## ~6 months of trade data

# BTC/USDT, ETH/USDT
# start = datetime.datetime(year=2017, month=9, day=1, hour=1)

# ETH, LTC
# datetime.datetime(year=2017, month=7, day=15, hour=1)

# NEO
# start = datetime.datetime(year=2017, month=8, day=1, hour=1)

# EOS, DASH, BCH
# datetime.datetime(year=2017, month=11, day=1, hour=1)

# XRP, XMR, ZEC, ADA
# datetime.datetime(year=2017, month=12, day=1, hour=1)

## GDAX

# Fetch order broken for GDAX
# https://github.com/ccxt/ccxt/blob/master/python/ccxt/gdax.py#L309
exchange = load_exchange('poloniex')
a = Asset(coins.BTC, coins.USD)
b = Asset(coins.XMR, coins.BTC)
start = datetime.datetime(year=2017, month=1, day=1, hour=0)
end = datetime.datetime(year=2018, month=1, day=2, hour=0)
from punisher.utils import dates
from punisher.trading.trade import Trade
from punisher.feeds import trade_feed

In [14]:
trades = exchange.fetch_public_trades(b, start, end)

ExchangeError: poloniex {"error":"Please specify a time window of no more than 1 month."}

In [ ]:
df = trade_feed.load_trades(exchange.id, b, start)

In [18]:
df.tail()

,exchange_id,exchange_order_id,price,quantity,trade_time,fee,side,symbol
id,,,,,,,,
7257429,poloniex,NaN,0.01403,0.008867,2017-01-01 23:59:38+00:00,0.0,SELL,XMR/BTC
7257430,poloniex,NaN,0.01403,0.008867,2017-01-01 23:59:40+00:00,0.0,SELL,XMR/BTC
7257431,poloniex,NaN,0.01403,0.008867,2017-01-01 23:59:44+00:00,0.0,SELL,XMR/BTC
7257432,poloniex,NaN,0.01403,0.008867,2017-01-01 23:59:47+00:00,0.0,SELL,XMR/BTC
7257433,poloniex,NaN,0.01403,0.008867,2017-01-01 23:59:50+00:00,0.0,SELL,XMR/BTC


## Asyncio

* https://hackernoon.com/asyncio-for-the-working-python-developer-5c468e6e2e8e

In [86]:
import asyncio 
import functools

async def yo(num):
    return {'num': num}

async def foo(a, b, c):
    print('Running in foo')
    print(a, b, c)
    await asyncio.sleep(5)
    print('Explicit context switch to foo again')
    return 10

async def bar():
    print('Explicit context to bar')
    #await asyncio.sleep(0)
    print('Implicit context switch back to bar')
    num = await yo(5)
    return num

loop = asyncio.new_event_loop()
a,b,c = 'abc'
tasks = [loop.create_task(foo(a,b,c)), loop.create_task(bar())]
wait_tasks = asyncio.wait(tasks)
loop.run_until_complete(wait_tasks)
#loop.run_forever()
#loop.close()

Running in foo
a b c
Explicit context to bar
Implicit context switch back to bar
Explicit context switch to foo again


({<Task finished coro=<bar() done, defined at <ipython-input-86-f6f863d6a3a2>:14> result={'num': 5}>,
  <Task finished coro=<foo() done, defined at <ipython-input-86-f6f863d6a3a2>:7> result=10>},
 set())

In [ ]:
loop.run_forever()

In [59]:
import time

start = time.time()


def tic():
    return 'at %1.1f seconds' % (time.time() - start)


async def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('gr1 started work: {}'.format(tic()))
    await asyncio.sleep(3)
    print('gr1 ended work: {}'.format(tic()))


async def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('gr2 started work: {}'.format(tic()))
    await asyncio.sleep(3)
    print('gr2 Ended work: {}'.format(tic()))


async def gr3():
    print("Let's do some stuff while the coroutines are blocked, {}".format(tic()))
    await asyncio.sleep(1)
    print("Done!")


ioloop = asyncio.new_event_loop()
tasks = [
    ioloop.create_task(gr1()),
    ioloop.create_task(gr2()),
    ioloop.create_task(gr3())
]
ioloop.run_until_complete(asyncio.wait(tasks))
ioloop.close()

gr1 started work: at 0.0 seconds
gr2 started work: at 0.0 seconds
Let's do some stuff while the coroutines are blocked, at 0.0 seconds
Done!
gr1 ended work: at 3.0 seconds
gr2 Ended work: at 3.0 seconds


In [62]:
import urllib.request
import aiohttp

URL = 'https://api.github.com/events'
MAX_CLIENTS = 3


def fetch_sync(pid):
    print('Fetch sync process {} started'.format(pid))
    start = time.time()
    response = urllib.request.urlopen(URL)
    datetime = response.getheader('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format(
        pid, datetime, time.time() - start))

    return datetime


async def fetch_async(pid):
    print('Fetch async process {} started'.format(pid))
    start = time.time()
    response = await aiohttp.request('GET', URL)
    datetime = response.headers.get('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format(
        pid, datetime, time.time() - start))

    response.close()
    return datetime


def synchronous():
    start = time.time()
    for i in range(1, MAX_CLIENTS + 1):
        fetch_sync(i)
    print("Process took: {:.2f} seconds".format(time.time() - start))


async def asynchronous():
    start = time.time()
    tasks = [asyncio.ensure_future(
        fetch_async(i)) for i in range(1, MAX_CLIENTS + 1)]
    await asyncio.wait(tasks)
    print("Process took: {:.2f} seconds".format(time.time() - start))


print('Synchronous:')
synchronous()

print('Asynchronous:')
ioloop = asyncio.new_event_loop()
ioloop.run_until_complete(asynchronous())
ioloop.close()

Synchronous:
Fetch sync process 1 started
Process 1: Sat, 24 Feb 2018 04:09:16 GMT, took: 0.35 seconds
Fetch sync process 2 started
Process 2: Sat, 24 Feb 2018 04:09:17 GMT, took: 0.32 seconds
Fetch sync process 3 started
Process 3: Sat, 24 Feb 2018 04:09:17 GMT, took: 0.34 seconds
Process took: 1.02 seconds
Asynchronous:
Fetch async process 1 started
Fetch async process 2 started
Fetch async process 3 started


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa58e5aafd0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa58e5bf6d8>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa58e5aa438>


Process 1: Sat, 24 Feb 2018 04:09:17 GMT, took: 0.34 seconds
Process 2: Sat, 24 Feb 2018 04:09:17 GMT, took: 0.34 seconds
Process 3: Sat, 24 Feb 2018 04:09:17 GMT, took: 0.35 seconds
Process took: 0.37 seconds


In [78]:
from collections import namedtuple
import time
import asyncio
import aiohttp
import traceback

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('borken', 'http://no-way-this-is-going-to-work.com/json', 'ip'),
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'this-is-not-an-attr'),
)


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    try:
        response = await aiohttp.request('GET', service.url)
    except:
        return '{} is unresponsive'.format(service.name)

    json_response = await response.json()
    ip = json_response[service.ip_attr]

    response.close()
    return '{} finished with result: {}, took: {:.2f} seconds'.format(
        service.name, ip, time.time() - start)


async def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    done, _ = await asyncio.wait(futures)

    for future in done:
        try:
            print(future.result())
        except:
            print("Unexpected error: {}".format(traceback.format_exc()))


ioloop = asyncio.new_event_loop()
ioloop.run_until_complete(asynchronous())
ioloop.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa58ebf2550>


Fetching IP from ipify
Fetching IP from borken
Fetching IP from ip-api
Fetching IP from ipify


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa58ebf20f0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa58dd41d30>


Unexpected error: Traceback (most recent call last):
  File "<ipython-input-78-ca4b55bef5b5>", line 40, in asynchronous
    print(future.result())
  File "<ipython-input-78-ca4b55bef5b5>", line 27, in fetch_ip
    ip = json_response[service.ip_attr]
KeyError: 'this-is-not-an-attr'

ipify finished with result: 24.5.150.30, took: 0.37 seconds
ipify finished with result: 24.5.150.30, took: 0.37 seconds
borken is unresponsive


## Fetch Ticker

* https://github.com/ccxt/ccxt/blob/master/examples/py/async-tickers.py

In [ ]:
async def fetch_ticker(exchange, asset):
    ticker = exchange.fetch_ticker(asset)
    return ticker

def fetch_tickers(exchange, assets):
    _ = exchange.get_markets()
    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(fetch_ticker(exchange, asset)) for asset in assets]
    results = loop.run_until_complete(asyncio.wait(tasks))
    #loop.close()
    return list(results[0])

In [ ]:
asset = Asset(coins.ETH, coins.BTC)
assets = [asset, Asset(coins.LTC, coins.BTC)]
_ = exchange.fetch_ticker(asset)

In [ ]:
res = fetch_tickers(exchange, assets)
res[0].result()

## Fetch Order Book

In [26]:
async def fetch_order_book(exchange, asset):
    ticker = exchange.fetch_order_book(asset)
    return ticker

def fetch_tickers(exchange, assets):
    _ = exchange.get_markets()
    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(fetch_ticker(exchange, asset)) for asset in assets]
    results = loop.run_until_complete(asyncio.wait(tasks))
    #loop.close()
    return list(results[0])

In [ ]:
exchange = load_exchange('gdax')
asset = Asset(coins.ETH, coins.BTC)
assets = [asset, Asset(coins.LTC, coins.BTC)]

In [ ]:
# Normal Order book
book = exchange.fetch_order_book(asset)
len(book['asks'])

In [29]:
# Raw Order book (un-aggregated)
exchange = load_exchange('gdax')
asset = Asset(coins.ETH, coins.USD)
assets = [asset]
book = exchange.fetch_raw_order_book(asset, level=2)
len(book['bids'])

50

## Exchange

In [ ]:
# Public informaiton
exchanges = [ex_cfg.PAPER, ex_cfg.BINANCE, ex_cfg.GDAX, ex_cfg.POLONIEX]
a = Asset(coins.ETH, coins.BTC)
ms=[]
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    ms.append(exchange.get_markets())
    exchange.fetch_ohlcv(a, Timeframe.FIFTEEN_MIN, datetime.datetime.utcnow())
    exchange.fetch_order_book(a)
    exchange.fetch_public_trades(a)
    exchange.fetch_ticker(a)

In [ ]:
# Public informaiton
exchanges = [ex_cfg.PAPER, ex_cfg.BINANCE, ex_cfg.GDAX, ex_cfg.POLONIEX]
a = Asset(coins.ETH, coins.BTC)
ex =  ex_cfg.BINANCE
print("Exchange", ex)
exchange = load_exchange(ex)
exchange.get_markets()
print(exchange.fetch_order_book(a, params={"limit":10}))
exchange.fetch_public_trades(a)
exchange.fetch_ticker(a)

In [ ]:
# Account Information
exchanges = [ex_cfg.PAPER, ex_cfg.BINANCE, ex_cfg.GDAX]
a = Asset(coins.ETH, coins.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    b = exchange.fetch_balance()
    exchange.fetch_balance()
    exchange.fetch_orders(a)
    exchange.fetch_open_orders(a)
    exchange.fetch_closed_orders(a)

In [ ]:
exchange = load_exchange(ex_cfg.PAPER) # ex_cfg.BINANCE
a = Asset(coins.ETH, coins.BTC)

# Market BUY
print("Exchange", exchange.id)
order = exchange.create_market_buy_order(a, .01)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
order

In [ ]:
# Check Order
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
found_order = exchange.fetch_order(order.ex_order_id, a.symbol)
found_order

In [ ]:
# Sell all remaining quantity
curr_balance = exchange.fetch_balance().get(a.base)
print(a.base, curr_balance[BalanceType.TOTAL])
order = exchange.create_market_sell_order(a, curr_balance[BalanceType.TOTAL])
order

In [ ]:
exchange.fetch_balance().get(a.base), exchange.fetch_balance().get(a.quote)

### Dates

In [ ]:
print(Timeframe.FIVE_MIN.delta)
print(Timeframe.FIVE_MIN.id)

### OHLCV Data

In [ ]:
base = coins.ETH
quote = coins.BTC
exchange = load_exchange(ex_cfg.PAPER)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [coins.ETH, coins.LTC] ]
timeframe = Timeframe.THIRTY_MIN
start = datetime.datetime(year=2017, month=9, day=1)
end = None
#exchange = load_exchange(ex_cfg.BINANCE)

In [ ]:
# Single Coin
df = ohlcv_feed.fetch_and_save_asset(exchange, asset, timeframe, start, end)
df

In [ ]:
# Load from File
fpath = ohlcv_feed.get_ohlcv_fpath(asset, exchange.id, timeframe)
df = ohlcv_feed.load_asset(fpath)
df.head()

In [ ]:
# Multiple Coins
exchanges = [exchange]
assets = [ Asset(coin, quote) for coin in [coins.ETH, coins.LTC] ]
ohlcv_feed.download_ohlcv(exchanges, assets, timeframe, start, end, update=False)
df = ohlcv_feed.load_multiple_assets([exchange.id], assets, timeframe, start)
df.head()

In [ ]:
# Get Benchmark Assets
exchanges = [ex_cfg.PAPER, ex_cfg.BINANCE, ex_cfg.GDAX, ex_cfg.POLONIEX]
for ex in exchanges:
    ex = load_exchange(ex)
    print(ex.id, ohlcv_feed.get_benchmark_asset(ex).symbol)

### DataStore

In [ ]:
experiment_name = 'store_testing'
fname = 'ohlcv.csv'
store = FileStore(os.path.join(cfg.DATA_DIR, experiment_name))

In [ ]:
exchange = load_exchange(ex_cfg.BINANCE)
asset = Asset(coins.ETH, coins.BTC)
timeframe = Timeframe.FIFTEEN_MIN
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)
df = ohlcv_feed.fetch_and_save_asset(exchange, asset, timeframe, start, end)

In [ ]:
# DataFrame --> CSV
store.df_to_csv(df, fname)
df = store.csv_to_df(fname, index='epoch')
df.head()

In [ ]:
# DataFrame --> JSON
store.df_to_json(df, fname)
df = store.json_to_df(fname, index='epoch')
df.head()

In [ ]:
# JSON
dct = {
    'sample': 1,
    'time': datetime.datetime.utcnow()
}
store.save_json(fname, dct)
dct = store.load_json(fname)
dct

### Data Feed

In [ ]:
default_assets = [Asset(coins.ETH, coins.BTC)]
def get_test_live_feed(exchange_ids, assets=None):
    assets = assets if assets is not None else default_assets
    exchanges = [load_exchange(id_) for id_ in exchange_ids]
    timeframe = Timeframe.FIFTEEN_MIN
    start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
    end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)
    feed = OHLCVExchangeFeed(exchanges, assets, timeframe, start, end=None)
    return feed

def get_test_csv_feed(exchange_id, assets=None):
    assets = assets if assets is not None else default_assets
    start = datetime.datetime(year=2018, month=1, day=9)
    end = datetime.datetime(year=2018, month=1, day=12)
    timeframe = Timeframe.THIRTY_MIN
    exchange = load_exchange(exchange_id)
    ohlcv_feed.fetch_and_save_asset(exchange, asset, timeframe, start, end)
    feed = OHLCVFileFeed([exchange_id], assets, timeframe, start, end)
    return feed

In [ ]:
# FileFeed - Single Asset + Exchange
ex_id = ex_cfg.BINANCE
asset = Asset(coins.ETH, coins.BTC)
csv_feed = get_test_csv_feed(ex_id, [asset])

data = csv_feed.next()
print(data.get('utc'), data.get('close', asset.symbol, ex_id))
peek_data = csv_feed.peek()
print(data.get('utc'), data.get('close', asset.symbol, ex_id))

# Access all rows in history
csv_feed.history().df
    
peek_data = csv_feed.peek()
peek_data.df

In [ ]:
# ExchangeFeed - Single Asset +  Exchange 
asset = Asset(coins.LTC, coins.BTC)
ex_id = ex_cfg.BINANCE

live_feed = get_test_live_feed([ex_id], assets=[asset])

# Grab 1 row at a time
for i in range(1):
    data = live_feed.next()
    print(data.get('utc'), data.get('close', asset.symbol, ex_id))    

peek_data = live_feed.peek()
print(peek_data.get('utc'), peek_data.get('close', asset.symbol, ex_id))

# Access all rows in history
live_feed.history().df

In [ ]:
# ExchangeFeed - Single Asset + Multiple Exchanges
assets = [Asset(coins.ETH, coins.BTC)]
exchange_ids = [ex_cfg.BINANCE, ex_cfg.PAPER]

live_feed = get_test_live_feed(exchange_ids, assets)

peek_data = live_feed.peek()
print(peek_data.get('utc'), peek_data.get('close', assets[0].symbol, ex_cfg.BINANCE))

# Access all rows in history
print(live_feed.history().df)
print(live_feed.history().df['open_ETH/BTC_binance'])

In [ ]:
# ExchangeFeed - Multiple Assets + Multiple Exchanges
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
exchange_ids = [ex_cfg.BINANCE, ex_cfg.GDAX]

live_feed = get_test_live_feed(exchange_ids, assets)

peek_data = live_feed.peek()
print(peek_data.get('utc'), peek_data.get('close', assets[1].symbol, ex_cfg.BINANCE))

# Access all rows in history
print(live_feed.history().df)
print(live_feed.history().df['open_ETH/BTC_binance'])

### Currency Conversion

In [ ]:
# ExchangeFeed - Multiple Assets + Multiple Exchanges
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
exchange_ids = [ex_cfg.BINANCE, ex_cfg.GDAX]
live_feed = get_test_live_feed(exchange_ids, assets)

In [ ]:
data = live_feed.next(refresh=False)
data = live_feed.next(refresh=False)
data = live_feed.history()
data.df

In [ ]:
rates = {}
for cash_coin in [coins.USD, coins.BTC, coins.USDT]:
    for ex_id in exchange_ids:
        for asset in assets:
            res = ohlcv_feed.get_exchange_rate(
                data.df, asset.quote, cash_coin, ex_id)
            rates[asset.symbol + ex_id + cash_coin] = res
for k,v in rates.items():
    print(k,':', v)

In [ ]:
vals = {}
for cash_coin in [coins.USD, coins.BTC, coins.USDT]:
    for ex_id in exchange_ids:
        for asset in assets:
            res = ohlcv_feed.get_cash_value(
                data.df, 'close', asset, ex_id, cash_coin)
            vals[asset.symbol + ex_id + cash_coin] = res
for k,v in vals.items():
    print(k,':', v)

### Exchange Data Providers

In [ ]:
ex_id = ex_cfg.BINANCE
asset = Asset(coins.ETH, coins.BTC)
csv_feed = get_test_csv_feed(ex_id, [asset])
csv_feed.next()
dp = FeedExchangeDataProvider(csv_feed, ex_id)
start = datetime.datetime.utcnow()
print(dp.fetch_order_book(asset))
print(dp.fetch_public_trades(asset))
print(dp.fetch_ohlcv(asset, Timeframe.ONE_DAY, start))
print(dp.fetch_ticker(asset))
print(dp.get_markets())

In [ ]:
asset = Asset(coins.ETH, coins.BTC)
exchange = load_exchange(ex_cfg.BINANCE)
dp = CCXTExchangeDataProvider(exchange)
print(dp.fetch_order_book(asset)['bids'][:1])
print(dp.fetch_public_trades(asset)[:1])
print(dp.fetch_ohlcv(asset, Timeframe.ONE_DAY, start)[:1])
print(dp.fetch_ticker(asset))
print(dp.get_markets()['ETH/BTC'])

### OrderType

In [ ]:
assert OrderType.from_type_side('limit','buy') == OrderType.LIMIT_BUY
assert OrderType.from_type_side('limit','sell') == OrderType.LIMIT_SELL
assert OrderType.from_type_side('market','buy') == OrderType.MARKET_BUY
assert OrderType.from_type_side('market','sell') == OrderType.MARKET_SELL
o = OrderType.LIMIT_BUY
assert o.type == 'limit' and o.side == 'buy'

In [ ]:
assert OrderType.LIMIT_BUY in OrderType.buy_types()
assert OrderType.MARKET_BUY in OrderType.buy_types()
assert OrderType.LIMIT_SELL in OrderType.sell_types()
assert OrderType.MARKET_SELL in OrderType.sell_types()

In [ ]:
assert OrderType.LIMIT_BUY.is_buy()
assert OrderType.MARKET_BUY.is_buy()
assert OrderType.LIMIT_SELL.is_sell()
assert OrderType.MARKET_SELL.is_sell()
OrderType.LIMIT_BUY.name, OrderType.LIMIT_BUY.value

### Order

In [ ]:
asset = Asset(coins.LTC, coins.USDT)
order = Order(
    exchange_id=ex_cfg.PAPER, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
Order.from_dict(order.to_dict())

### Order Manager

In [ ]:
asset = Asset(coins.LTC, coins.BTC)
exchange = load_exchange(ex_cfg.PAPER)
exchange.balance = Balance(coins.BTC, 5.0)
o1 = Order(exchange.id, asset, price=.01, quantity=1.0, 
           order_type=OrderType.LIMIT_BUY)
o2 = Order(exchange.id, asset, price=.01, quantity=1.0, 
           order_type=OrderType.LIMIT_SELL)
orders = {
    o1.id: o1,
    o2.id: o2
}
orders,exchange.fetch_balance(), exchange.fetch_ticker(asset)['ask']

In [ ]:
order = order_manager.place_order(exchange, orders[o1.id])
orders[o1.id] = order
orders,exchange.fetch_balance()

In [ ]:
resp = order_manager.place_order(exchange, orders[o2.id])
orders[o2.id] = resp
orders,exchange.balance

In [ ]:
limit_buy = order_manager.build_limit_buy_order(exchange, asset, quantity=1.0, price=.1)
limit_sell = order_manager.build_limit_sell_order(exchange, asset, quantity=1.0, price=.1)
market_buy = order_manager.build_market_buy_order(exchange, asset, quantity=1.0)
market_sell = order_manager.build_market_sell_order(exchange, asset, quantity=1.0)
orders = [limit_buy, limit_sell, market_buy, market_sell]
orders

In [ ]:
results = []
for order in orders:
    res = order_manager.place_order(exchange, order)
    results.append(res)
results

In [ ]:
exchange.fetch_balance()

In [ ]:
exchange.fetch_orders(asset)

In [ ]:
exchange.fetch_order('32687cf2c22645fbbd2d639be8a76256', asset)

In [ ]:
updated_orders = exchange.fetch_orders(asset)
ex_order_ids = [order.ex_order_id for order in updated_orders]
print(ex_order_ids)
updated_orders

In [ ]:
ex_orders = order_manager.get_orders(exchange, ex_order_ids, assets=asset)
print(ex_orders)

In [ ]:
print("OPEN", order_manager.get_open_orders(ex_orders))
print("CANCELED", order_manager.get_canceled_orders(ex_orders))
print("FILLED", order_manager.get_filled_orders(ex_orders))

### Position

In [ ]:
asset = Asset(coins.LTC, coins.BTC)
pos = Position(asset, quantity=1, cost_price=250.0)
print("Asset", pos.asset.to_dict())
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(-1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Go short
pos.update(-2, 150)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Close the short for a LOSS and go long (price went up, we had to pay $300 to cover our short)
# then we bought another share because we're newbs
pos.update(2, 300)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

In [ ]:
"""
Assume that an investor made the following consecutive fund purchases in a taxable account: 1,500 shares at $20, 1,000 shares at $10 and 1,250 shares at $8. The investor’s average cost basis is calculated by dividing $50,000/3,750 shares. The average cost is $13.33.

Suppose the investor then sells 1,000 shares of the fund at $19.

Gain/loss using average cost basis: ($19 - $13.33) x 1,000 shares = $5,667
"""

asset = Asset(coins.LTC, coins.BTC)
pos = Position(asset, quantity=1500, cost_price=20.0)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1000, txn_price=10)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1250, txn_price=8)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

last_cost = pos.cost_price
pos.update(txn_quantity=-1000, txn_price=19)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
print("Profit", (19 - last_cost) * 1000)

In [ ]:
Position.from_dict(pos.to_dict()).to_dict()

### PerformanceTracker

In [ ]:
asset = Asset(coins.BTC, coins.USD)
perf = PerformanceTracker(starting_cash=5000, timeframe=Timeframe.ONE_MIN)
perf.to_dict()

In [ ]:
# Buy 1 BTC for $1000
pos1 = Position(asset, quantity=1, cost_price=1000.0)
positions = [pos1]
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased $100
Position Return
    Return = .10
    PnL = $100
Cumulative Return
    Return = $100 / $5000 = .02
    PnL = $100
"""
pos1.latest_price = 1100
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased again $400
Position Return
    Return = .5
    PnL = $500
Cumulative Return
    Return = $500 / $5000 = .1
    PnL = $500
"""
pos1.latest_price = 1500
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
# Buy 1 ETH for $500
asset = Asset(coins.ETH, coins.USD)
pos2 = Position(asset, quantity=1, cost_price=500.0)
positions.append(pos2)
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of ETH decreased $100
Position Return
    Return = -100/500
    PnL = -100
Cumulative Return
    Return = 400 / 5000 = .08
    PnL = 400
"""
pos2.latest_price = 400
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
PerformanceTracker.from_dict(perf.to_dict())

### Portfolio

In [ ]:
starting_cash = 5000
cash_currency = coins.BTC
exchange = load_exchange(ex_cfg.PAPER)
perf = PerformanceTracker(starting_cash, Timeframe.ONE_MIN)
portfolio = Portfolio(cash_currency, starting_cash, perf)

In [ ]:
# Buy 1 BTC for $1000
asset = Asset(coins.BTC, coins.USD)
latest_prices = {asset.symbol: .14}
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1000., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

In [ ]:
portfolio.update(datetime.datetime.utcnow(), [order], latest_prices)
portfolio

In [ ]:
# Buy more BTC, price has risen $500
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
portfolio.update(datetime.datetime.utcnow(), [order], latest_prices)
portfolio

In [ ]:
# Sell all BTC at profit
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=2, 
    order_type=OrderType.LIMIT_SELL
)
portfolio.update(datetime.datetime.utcnow(), [order], latest_prices)
portfolio

In [ ]:
Portfolio.from_dict(portfolio.to_dict())

### Record

In [ ]:
asset = Asset(coins.ETH, coins.BTC)
feed = get_test_csv_feed(ex_cfg.BINANCE, [asset])
starting_cash = 5000
perf = PerformanceTracker(starting_cash, Timeframe.THIRTY_MIN)
portfolio = Portfolio(coins.BTC, starting_cash, perf)

metrics = {
    'SMA': [1, 2, 3, 4],
    'RSI': [3, 4, 2, 1]
}

o1 = Order(ex_cfg.PAPER, Asset(coins.ETH, coins.BTC), 5.0, 100.0, OrderType.LIMIT_BUY)
o2 = Order(ex_cfg.PAPER, Asset(coins.LTC, coins.BTC), 5.0, 100.0, OrderType.LIMIT_BUY)
orders = {
    o1.id: o1,
    o2.id: o2
}

In [ ]:
root_dir = os.path.join(cfg.DATA_DIR, 'default_backtest')
store = FileStore(root_dir)

record = Record(config={}, portfolio=portfolio, balance=Balance(), store=store)
record.orders = orders
record.metrics = metrics
record.ohlcv = feed.history().ohlcv_df
record.save()
record = Record.load(root_dir)

In [ ]:
feed.next()
feed.next()
record.ohlcv = feed.history().ohlcv_df
record.save()
record = Record.load(root_dir)

In [ ]:
record.add_ohlcv(feed.next())

In [ ]:
feed.next().row(0)

In [ ]:
record.ohlcv

In [ ]:
print(record.config)
print(record.metrics)
print(record.balance)
print(record.orders)
print(record.ohlcv.head()['utc'])
print(record.portfolio)

### Context

In [ ]:
# From Objects
asset = Asset(coins.ETH, coins.BTC)
starting_cash = 5000
exchange = load_exchange(ex_cfg.PAPER)
store = FileStore(os.path.join(cfg.DATA_DIR, 'default'))
perf = PerformanceTracker(starting_cash, Timeframe.THIRTY_MIN)
portfolio = Portfolio(starting_cash, perf)
feed = get_test_csv_feed(ex_cfg.BINANCE, [asset])
config = {'experiment':'default'}
record = Record(config=config, 
                portfolio=portfolio, 
                balance=Balance(), 
                store=store)

In [ ]:
ctx = Context(exchange, feed, record)
print(ctx.record.config)
print(ctx.record.metrics)
print(ctx.record.balance)
print(ctx.record.orders)
print(ctx.record.ohlcv)
print(ctx.record.portfolio)

### ChartDataProviders

In [ ]:
root = os.path.join(cfg.DATA_DIR, 'default')
rp = RecordChartDataProvider(root)

In [ ]:
# TODO: THIS WILL FAIL UNLESS A NON-EMPTY PORTFOLIO EXISTS IN THE ABOVE DIRECTORY
print("OHLCV\n", rp.get_ohlcv(coins.BTC))
print("\nPERFORMANCE\n", rp.get_performance())
print("\nRETURNS\n", rp.get_returns(coins.BTC, ex_cfg.PAPER))
print("\nPNL\n", rp.get_pnl(coins.BTC, ex_cfg.PAPER))

print("\nBALANCE\n", rp.get_balance())
print("\nBALANCE_DICT\n", rp.get_balance_dct())
print("\nPOSITIONS\n", rp.get_positions())
print("\nPOSITIONS_DICT\n", rp.get_positions_dct())

print("\nORDERS\n", rp.get_orders())
print("\nORDERS_DICT\n", rp.get_orders_dct())
print("\nMETRICS\n", rp.get_metrics())

### Charts

In [ ]:
feed = get_test_csv_feed(ex_cfg.BINANCE, Asset(coins.ETH,coins.BTC))

In [ ]:
# We need to increment the feed to start getting history data
for i in range(10):
    _ = feed.next()

In [ ]:
punisher.utils.charts.plot_range(
    feed.history().ohlcv_df, start=None, end=None, 
    column_name='close_ETH/BTC_'+ex_cfg.BINANCE)

### Dash

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

periods = record.portfolio.perf.periods
df = pd.DataFrame([
    [p['end_time'], p['pnl']] for p in periods
], columns=['utc','pnl'])
df

In [ ]:
positions = pd.DataFrame([p.to_dict() for p in record.portfolio.positions])
dct = [p.to_dict() for p in record.portfolio.positions]
{p['asset']: p for p in dct}

In [ ]:
cols = ['coin', 'free', 'used', 'total']
balance = Balance()
coins = balance.currencies
dct = balance.to_dict()
df = pd.DataFrame(
    [[c, dct[c]['free'], dct[c]['used'], dct[c]['total']] for c in coins],
    columns=cols
)

df